### Vamos a probar la nueva data en español

In [1]:
import pandas as pd
import sys
import os
import re
sys.path.append(os.path.abspath('../scripts'))  #esto lo colocamos para que el python nos lea la ruta bien
from utils import process_null_values, tokenice_and_lemati # Importa la funcinn desde utils


In [ ]:
MultiLeng = pd.read_csv('../data/spanish/train.csv')
MultiLeng.describe()

In [ ]:
DataS = pd.read_csv('../data_process/DataSp1r.csv')

In [ ]:
MultiLeng.head()

### Eliminando fiilas innecesarias

In [ ]:
MultiLeng = MultiLeng.drop(MultiLeng[(MultiLeng['language'] != 'en') & (MultiLeng['language'] != 'es')].index)

In [ ]:
conteo_reseñas = MultiLeng.groupby('language').size()
conteo_reseñas

## Limpiando datos

In [ ]:
process_null_values(DataS)

## Eliminando columnas innecesarias

In [ ]:
MultiLeng.drop(['review_id'], axis=1, inplace=True)
MultiLeng.drop(['product_id'], axis=1, inplace=True)
MultiLeng.drop(['reviewer_id'], axis=1, inplace=True)
MultiLeng.drop(['product_category'], axis=1, inplace=True)
MultiLeng.drop(['Unnamed: 0'], axis=1, inplace=True)


In [ ]:
MultiLeng.head()

### divideremos la data en espaniol e ingles, por si acaso la de ingles no es de ayuda

In [ ]:
DataSpanish = MultiLeng[(MultiLeng['language'] == 'es')]
DataEnglish = MultiLeng[(MultiLeng['language'] == 'en')]


In [ ]:
DataSpanish.head()

## VISUALIZAR DANTIDAD DE DE LOS PUNTAJES

In [ ]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay(no hay de otra)
#eliminar muchos registros para balancear la data
condition = DataS['stars'] == 1
num_to_remove = 1000
rows_to_remove = DataS[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
DataS = DataS.drop(rows_to_remove.index)

DataS.describe()


In [ ]:
conteo_reseñas = DataS.groupby('stars').size()
conteo_reseñas

In [ ]:
DataSpanish['processed_text'] = DataSpanish['review_body'].apply(lambda x: re.sub(r'[^\w\sáéíóúñÁÉÍÓÚÑ]', '', x))

In [ ]:
DataS.head()

In [ ]:
# Procesar la columna review_body en lote
DataS['processed_text'] = tokenice_and_lemati(DataS['review_body'], process=2, language='es')

In [ ]:
DataS.to_csv('../data_process/DataSp1r.csv', index=False)

In [ ]:
from transformers import pipeline


sentiment_analyzer = pipeline(
    "text-classification", 
    model="nlptown/bert-base-multilingual-uncased-sentiment",  
    device=-1 
)

def analizar_sentimiento_beto(textos):
    resultados = []
    
    predicciones = sentiment_analyzer(textos, truncation=True)
    
    for pred in predicciones:
        label = pred["label"]
        
        if "5" in label:
            resultados.append("contento")  
        elif "4" in label:
            resultados.append("positivo")  
        elif "1" in label:
            resultados.append("enojado") 
        elif "2" in label:
            resultados.append("insatisfecho")
        elif "3" in label:
            resultados.append("neutral")
        else:
            resultados.append("neutral")
    
    return resultados


In [ ]:
data_muestra = DataS.head(1000)

In [ ]:
textos = data_muestra["review_body"].tolist()

In [ ]:
sentimientos = analizar_sentimiento_beto(textos)

In [ ]:
data_muestra["sentiment"] = sentimientos

In [ ]:
data_muestra.head()

In [ ]:
data_muestra.to_csv('../data_process/Muestra.csv', index=False)


### Union de los 2 datasets

In [2]:
#unir los 3 dataset
dataset1 = pd.read_csv('../data_process/45kSpanish2.csv')
dataset2 = pd.read_csv('../data_process/50kSpanish.csv')


In [3]:
df_unido = pd.concat([dataset1, dataset2,], axis=0, ignore_index=True)

In [5]:
df_unido.drop(['review_body'], axis=1, inplace=True)

In [7]:
df_unido.head()

,stars,language,processed_text,sentiment
0,3,es,funciona exactamente marca caro,enojado
1,3,es,llevan no sé funcionara,enojado
2,3,es,máquina mejor poco ruido cortando perfecta muy...,contento
3,3,es,meses funcionó muy protege teléfono golpes fun...,insatisfecho
4,3,es,caja bonito,contento


In [40]:
conteo_sentiment = df_unido.groupby('sentiment').size()
conteo_sentiment

sentiment
contento        17670
enojado         16858
insatisfecho    17315
neutral         14082
positivo        10421
dtype: int64

In [39]:
#eliminar muchos registros para balancear la data
condition = df_unido['sentiment'] == 'enojado'
num_to_remove = 1800
rows_to_remove = df_unido[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
df_unido = df_unido.drop(rows_to_remove.index)

df_unido.describe()

,stars
count,76346.000000
mean,2.970660
std,1.465102
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [12]:
df_unido.to_csv('../data_process/SpanishData.csv', index=False)

### Cargar datos en Ingles 

In [14]:
datasetIngles = pd.read_csv('../data_process/FinalDataTrainer.csv')

In [15]:
datasetIngles.describe()

,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [48]:
conteo_sentiment = datasetIngles.groupby('sentiment').size()
conteo_sentiment

sentiment
contento        13101
enojado         14204
insatisfecho    13916
neutral          9015
positivo        17636
dtype: int64

In [24]:
datasetIngles['language'] = 'en'

In [47]:
datasetIngles.describe()

,score
count,54399.000000
mean,3.224085
std,1.516618
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [33]:
#eliminar muchos registros para balancear la data
condition = datasetIngles['sentiment'] == 'negativo'
num_to_remove = 583
rows_to_remove = datasetIngles[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
datasetIngles = datasetIngles.drop(rows_to_remove.index)

datasetIngles.describe()

,score
count,54399.000000
mean,3.224085
std,1.516618
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [49]:
datasetIngles.to_csv('../data_process/dataInglesTr.csv', index=False)

In [29]:
condition = datasetIngles['sentiment'] == 'negativo'
num_to_modify = 3600

# Obtener los índices de las filas a modificar
rows_to_modify = datasetIngles[condition].head(num_to_modify).index

# Cambiar el valor en la columna deseada
datasetIngles.loc[rows_to_modify, 'sentiment'] = 'insatisfecho'  # Cambia 'neutral' por el valor que necesites

datasetIngles.describe()

,score
count,58982.000000
mean,3.215795
std,1.516760
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,5.000000


In [41]:
df_final = pd.concat([datasetIngles, df_unido,], axis=0, ignore_index=True)

In [46]:
df_final.describe()

,score,stars
count,54399.000000,76346.000000
mean,3.224085,2.970660
std,1.516618,1.465102
min,1.000000,1.000000
25%,2.000000,2.000000
50%,3.000000,3.000000
75%,5.000000,4.000000
max,5.000000,5.000000


In [45]:
df_final.to_csv('../data_process/FullData.csv', index=False)